In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

In [ ]:
df_kernel = pd.read_csv("ParallelKernel.csv")
df_img = pd.read_csv("ParallelImg.csv")

In [ ]:
MAX_THREADS = 8

x_threads = range(1, MAX_THREADS + 1)

def get_result(df, num_thread, kernel_size, img_size = 150):
    data = df[(df["N_threads"] == num_thread) & (df["Kernel_size"] == kernel_size) & (df["dim_images"] == img_size)]
    mean_k = np.mean(data.iloc[1:]["time_k"])
    mean_e2e = np.mean(data.iloc[1:]["time_e2e"])
    return (mean_k, mean_e2e)

## End To End time SpeedUp

In [ ]:
sequential = get_result(df_kernel, 1, 3)[1]
plt.plot(x_threads, [sequential / get_result(df_kernel, i, 3)[1]  for i in x_threads], label = "Parallel Kernel", marker = "o")
plt.plot(x_threads, [sequential / get_result(df_img, i, 3)[1]  for i in x_threads], label = "Parallel Image", marker = "o")

plt.title("Completition Time, Images 150x150, Box Blur 3x3")
plt.ylabel("SpeedUp")
plt.xlabel("Num Threads")
plt.grid()
plt.legend()
plt.savefig("img/end2end_3x3.png")

In [ ]:
sequential = get_result(df_kernel, 1, 7)[1]
plt.plot(x_threads, [sequential / get_result(df_kernel, i, 7)[1]  for i in x_threads], label = "Parallel Kernel", marker = "o")
plt.plot(x_threads, [sequential / get_result(df_img, i, 7)[1]  for i in x_threads], label = "Parallel Image", marker = "o")

plt.title("Completition Time, Images 150x150, Box Blur 7x7")
plt.ylabel("SpeedUp")
plt.xlabel("Num Threads")
plt.grid()
plt.legend()
plt.savefig("img/end2end_7x7.png")

## Kernel Time Application

In [ ]:
sequential = get_result(df_kernel, 1, 3)[0]
plt.plot(x_threads, [sequential / get_result(df_kernel, i, 3)[0]  for i in x_threads], label = "Parallel Kernel", marker = "o")
plt.plot(x_threads, [sequential / get_result(df_img, i, 3)[0]  for i in x_threads], label = "Parallel Image", marker = "o")

plt.title("Kernel Application, Images 150x150, Box Blur 3x3")
plt.ylabel("SpeedUp")
plt.xlabel("Num Threads")
plt.grid()
plt.legend()
plt.savefig("img/kernel_3x3.png")

In [ ]:
sequential = get_result(df_kernel, 1, 7)[0]
plt.plot(x_threads, [sequential / get_result(df_kernel, i, 7)[0]  for i in x_threads], label = "Parallel Kernel", marker = "o")
plt.plot(x_threads, [sequential / get_result(df_img, i, 7)[0]  for i in x_threads], label = "Parallel Image", marker = "o")

plt.title("Kernel Application, Images 150x150, Box Blur 7x7")
plt.ylabel("SpeedUp")
plt.xlabel("Num Threads")
plt.grid()
plt.legend()
plt.savefig("img/kernel_7x7.png")

# Cuda Results

In [ ]:
df_cuda = pd.read_csv("Cuda.csv")
df_cuda_tile = pd.read_csv("CudaTile.csv")
df_cuda_batch = pd.read_csv("CudaBatch.csv")
df_cuda_parallel_batch = pd.read_csv("CudaParallelBatch.csv")

In [ ]:
def get_omp_1024(threads, kernel_size):
    df = pd.read_csv("big_dataset.csv")
    data = df[(df["Kernel_size"] == kernel_size) & (df["N_threads"] == threads)]
    mean_kernel = data["time_k"].iloc[0]
    e2e = data["time_e2e"].iloc[0]

    return (mean_kernel, e2e)

## SpeedUp over Completion Time

## Images 150x150

In [ ]:
MAX_CUDA_THREADS = 4
x_threads = range(1, MAX_CUDA_THREADS + 1)
sequential = get_result(df_cuda, 1, 3)[1]
omp_top = get_result(df_img, 8, 3)[1]

plt.plot(x_threads, [sequential / get_result(df_cuda, i, 3)[1]  for i in x_threads], label = "Simple", marker = "o")
plt.plot(x_threads, [sequential / get_result(df_cuda_tile, i, 3)[1]  for i in x_threads], label = "Kernel Tiling", marker = "o")
plt.plot(x_threads, [sequential / get_result(df_cuda_batch, i, 3)[1]  for i in x_threads], label = "Batching", marker = "o")
plt.plot(x_threads, [sequential / get_result(df_cuda_parallel_batch, i, 3)[1]  for i in x_threads], label = "Parallel Batching", marker = "o")

plt.hlines(sequential / omp_top, 1, MAX_CUDA_THREADS, label = "OMP Top Perf.", color = "grey", ls='--')

plt.title("Completion Time (Cuda), Image 150x150, Box Blur 3x3")
plt.ylabel("SpeedUp")
plt.xlabel("Num Threads")
plt.legend()
plt.grid()
plt.savefig("img/cuda_150_3")

In [ ]:
sequential = get_result(df_cuda, 1, 7)[1]
omp_top = get_result(df_img, 8, 7)[1]

plt.plot(x_threads, [sequential / get_result(df_cuda, i, 7)[1]  for i in x_threads], label = "Simple", marker = "o")
plt.plot(x_threads, [sequential / get_result(df_cuda_tile, i, 7)[1]  for i in x_threads], label = "Kernel Tiling", marker = "o")
plt.plot(x_threads, [sequential / get_result(df_cuda_batch, i, 7)[1]  for i in x_threads], label = "Batching", marker = "o")
plt.plot(x_threads, [sequential / get_result(df_cuda_parallel_batch, i, 7)[1]  for i in x_threads], label = "Parallel Batching", marker = "o")

#plt.hlines(sequential / omp_top, 1, MAX_CUDA_THREADS, label = "OMP Top Perf.", color = "grey", ls='--')

plt.title("Completion Time (Cuda), Image 150x150, Box Blur 7x7")
plt.ylabel("SpeedUp")
plt.xlabel("Num Threads")
plt.legend()
plt.grid()
plt.savefig("img/cuda_150_7")

## Images 1024x1024

In [ ]:
sequential = get_result(df_cuda, 1, 3, img_size = 1024)[1]
omp_top = get_result(df_img, 8, 3)[1]

plt.plot(x_threads, [sequential / get_result(df_cuda, i, 3, img_size = 1024)[1]  for i in x_threads], label = "Simple", marker = "o")
plt.plot(x_threads, [sequential / get_result(df_cuda_tile, i, 3, img_size = 1024)[1]  for i in x_threads], label = "Kernel Tiling", marker = "o")
plt.plot(x_threads, [sequential / get_result(df_cuda_batch, i, 3, img_size = 1024)[1]  for i in x_threads], label = "Batching", marker = "o")
plt.plot(x_threads, [sequential / get_result(df_cuda_parallel_batch, i, 3, img_size = 1024)[1]  for i in x_threads], label = "Parallel Batching", marker = "o")

#plt.hlines(sequential / omp_top, 1, MAX_CUDA_THREADS, label = "OMP Top Perf.", color = "grey", ls='--')

plt.title("Completion Time (Cuda), Image 1024x1024, Box Blur 3x3")
plt.ylabel("SpeedUp")
plt.xlabel("Num Threads")
plt.legend()
plt.grid()
plt.savefig("img/cuda_1024_3")

In [ ]:
sequential = get_result(df_cuda, 1, 7, img_size = 1024)[1]
omp_top = get_result(df_img, 8, 3)[1]

plt.plot(x_threads, [sequential / get_result(df_cuda, i, 7, img_size = 1024)[1]  for i in x_threads], label = "Simple", marker = "o")
plt.plot(x_threads, [sequential / get_result(df_cuda_tile, i, 7, img_size = 1024)[1]  for i in x_threads], label = "Kernel Tiling", marker = "o")
plt.plot(x_threads, [sequential / get_result(df_cuda_batch, i, 7, img_size = 1024)[1]  for i in x_threads], label = "Batching", marker = "o")
plt.plot(x_threads, [sequential / get_result(df_cuda_parallel_batch, i, 7, img_size = 1024)[1]  for i in x_threads], label = "Parallel Batching", marker = "o")

#plt.hlines(sequential / omp_top, 1, MAX_CUDA_THREADS, label = "OMP Top Perf.", color = "grey", ls='--')

plt.title("Completion Time (Cuda), Image 1024x1024, Box Blur 7x7")
plt.ylabel("SpeedUp")
plt.xlabel("Num Threads")
plt.legend()
plt.grid()
plt.savefig("img/cuda_1024_7")

## Comparison with OpenMP

### Images 150x150

In [ ]:
sequential = get_result(df_img, 1, 3)[1]

plt.plot(x_threads, [sequential / get_result(df_cuda, i, 3)[1]  for i in x_threads], label = "Simple", marker = "o")
plt.plot(x_threads, [sequential / get_result(df_cuda_tile, i, 3)[1]  for i in x_threads], label = "Kernel Tiling", marker = "o")
plt.plot(x_threads, [sequential / get_result(df_cuda_batch, i, 3)[1]  for i in x_threads], label = "Batching", marker = "o")
plt.plot(x_threads, [sequential / get_result(df_cuda_parallel_batch, i, 3)[1]  for i in x_threads], label = "Parallel Batching", marker = "o")

#plt.hlines(sequential / omp_top, 1, MAX_CUDA_THREADS, label = "OMP Top Perf.", color = "grey", ls='--')

plt.title("Completion Time (Cuda), Image 150x150, Box Blur 3x3")
plt.ylabel("SpeedUp Over OpenMP")
plt.xlabel("Num Threads")
plt.legend()
plt.grid()
plt.savefig("img/cuda_over_omp_150_3")

In [ ]:
sequential = get_result(df_img, 1, 7)[1]

plt.plot(x_threads, [sequential / get_result(df_cuda, i, 7)[1]  for i in x_threads], label = "Simple", marker = "o")
plt.plot(x_threads, [sequential / get_result(df_cuda_tile, i, 7)[1]  for i in x_threads], label = "Kernel Tiling", marker = "o")
plt.plot(x_threads, [sequential / get_result(df_cuda_batch, i, 7)[1]  for i in x_threads], label = "Batching", marker = "o")
plt.plot(x_threads, [sequential / get_result(df_cuda_parallel_batch, i, 7)[1]  for i in x_threads], label = "Parallel Batching", marker = "o")

#plt.hlines(sequential / omp_top, 1, MAX_CUDA_THREADS, label = "OMP Top Perf.", color = "grey", ls='--')

plt.title("Completion Time (Cuda), Image 150x150, Box Blur 7x7")
plt.ylabel("SpeedUp Over OpenMP")
plt.xlabel("Num Threads")
plt.legend()
plt.grid()
plt.savefig("img/cuda_over_omp_150_7")

### Images 1024x1024

In [ ]:
sequential = get_omp_1024(1, 3)[1]
omp_top = get_omp_1024(4, 3)[1]

plt.plot(x_threads, [sequential / get_result(df_cuda, i, 3, img_size = 1024)[1]  for i in x_threads], label = "Simple", marker = "o")
plt.plot(x_threads, [sequential / get_result(df_cuda_tile, i, 3, img_size = 1024)[1]  for i in x_threads], label = "Kernel Tiling", marker = "o")
plt.plot(x_threads, [sequential / get_result(df_cuda_batch, i, 3, img_size = 1024)[1]  for i in x_threads], label = "Batching", marker = "o")
plt.plot(x_threads, [sequential / get_result(df_cuda_parallel_batch, i, 3, img_size = 1024)[1]  for i in x_threads], label = "Parallel Batching", marker = "o")

#plt.hlines(sequential / omp_top, 1, MAX_CUDA_THREADS, label = "OMP Top Perf.", color = "grey", ls='--')

plt.title("Completion Time (Cuda), Image 1024x1024, Box Blur 3x3")
plt.ylabel("SpeedUp Over OpenMP")
plt.xlabel("Num Threads")
plt.legend()
plt.grid()
plt.savefig("img/cuda_over_omp_1024_3")

In [ ]:
sequential = get_omp_1024(1, 7)[1]
omp_top = get_omp_1024(4, 7)[1]

plt.plot(x_threads, [sequential / get_result(df_cuda, i, 7, img_size = 1024)[1]  for i in x_threads], label = "Simple", marker = "o")
plt.plot(x_threads, [sequential / get_result(df_cuda_tile, i, 7, img_size = 1024)[1]  for i in x_threads], label = "Kernel Tiling", marker = "o")
plt.plot(x_threads, [sequential / get_result(df_cuda_batch, i, 7, img_size = 1024)[1]  for i in x_threads], label = "Batching", marker = "o")
plt.plot(x_threads, [sequential / get_result(df_cuda_parallel_batch, i, 7, img_size = 1024)[1]  for i in x_threads], label = "Parallel Batching", marker = "o")

#plt.hlines(sequential / omp_top, 1, MAX_CUDA_THREADS, label = "OMP Top Perf.", color = "grey", ls='--')

plt.title("Completion Time (Cuda), Image 1024x1024, Box Blur 7x7")
plt.ylabel("SpeedUp Over OpenMP")
plt.xlabel("Num Threads")
plt.legend()
plt.grid()
plt.savefig("img/cuda_over_omp_1024_7")